In [7]:
# REQUIRED INPUTS
REDLIST_KEY = input("IUCN Red List API key: ")
EBIRD_KEY = input("EBird API key: ")

In [3]:
import os
from ecoscape_layers import (
    LayerGenerator,
    warp,
    RedList,
    generate_resistance_table,
    in_habs,
    default_refinement_method,
)

In [4]:
DATA_PATH = "."

In [5]:
# Birds
species_list = ["acowoo", "stejay"]

# Paths
landcover_fn = os.path.join(DATA_PATH, "inputs", "test_terrain.tif")
out_landcover_fn = os.path.join(DATA_PATH, "outputs", "test_terrain_out.tif")
os.makedirs(os.path.dirname(out_landcover_fn), exist_ok=True)

# OPTIONAL INPUTS
crs = '''
PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]
'''
resolution = 300
resampling = "near"
bounds = (-2169323, 608763, -2156868, 615750)
padding = 0

In [5]:
# Reproject landcover, then initialize LayerGenerator instance.
warp(landcover_fn, out_landcover_fn, crs, resolution, bounds, padding, resampling)
layer_generator = LayerGenerator(out_landcover_fn, REDLIST_KEY, EBIRD_KEY)
redlist = RedList(REDLIST_KEY, EBIRD_KEY)

In [6]:
# Run the habitat layer generation process for each bird species.
for species_code in species_list:
    habitat_fn = os.path.join(DATA_PATH, "outputs", species_code, "habitat_test.tif")
    resistance_dict_fn = os.path.join(DATA_PATH, "outputs", species_code, "resistance_test.csv")
    range_fn = os.path.join(DATA_PATH, "outputs", species_code, "range_map_2022.gpkg")
    range_src = "ebird"

    # get IUCN Redlist Habitat data
    habitat_data = redlist.get_habitat_data(species_code, ebird_code=True if range_src == "ebird" else False)

    # add a custom habitat if acowoo
    if species_code == "acowoo":
        habitat_data[308] = {
            "code": "3.8",
            "habitat": "Shrubland - Mediterranean-type shrubby vegetation",
            "suitability": True,
            "season": "Resident",
            "majorimportance": True,
        }

    # define the refinement methods
    def forest_refinement_method(map_code: int, habitats: dict[int, dict[str, str | bool]]) -> float:
        if in_habs(map_code, ["forest"] + ([308] if species_code == "acowoo" else [])):
            return 0.0
        return default_refinement_method(map_code, habitats)

    # create the resistance csv
    generate_resistance_table(habitat_data, resistance_dict_fn, refinement_method=forest_refinement_method)

    # define the overrides for what is current habitat
    cur_hab = ["forest"] + ([308] if species_code == "acowoo" else [])

    # create the habitat layer
    layer_generator.generate_habitat(
        species_code,
        habitat_data,
        habitat_fn,
        range_fn,
        range_src,
        current_hab_overrides=cur_hab,
    )